In [32]:
import pandas as pd
import pyvi
from underthesea import word_tokenize
import re
import string

In [33]:
df = pd.read_csv('vnexpress.csv')
df.head()

,Unnamed: 0,id,title,author,updatetime,wordcount,publication,tags,content,category
0,0,4381821,Hai thứ cần kiểm tra ngay khi nhận phòng khách...,1278077694,1636102370,374,1636100385,"khách sạn, dọn phòng, giường, làm sạch","Nếu chăn, ga, gối và tấm trải giường nhàu nhĩ...",du-lich
1,1,4381773,Thảo Cầm Viên nhộn nhịp ngày đầu mở cửa trở lại,1700000208,1636102776,11,1636100861,"thảo cầm viên, sở thú, thảo cầm viên đóng cửa,...","Sáng 5/11, người dân xếp hàng ở quầy chờ mua ...",du-lich
2,2,4381741,Cúc Phương là vườn quốc gia hàng đầu châu Á 2021,1278077694,1636097710,356,1636086711,"du khách, du lịch, cúc phương, vườn quốc gia","Để giành danh hiệu trên, Cúc Phương đã vượt q...",du-lich
3,3,4376293,Cuộc sống không thể thiếu chocolate ở Bỉ,1700000475,1636098959,692,1636083674,"chocolate, chocolate bỉ, du lịch bỉ, du lịch b...","""Người Bỉ khó có thể tồn tại mà không có choc...",du-lich
4,4,4381205,Bánh canh hẹ Phú Yên ở Sài Gòn,1700000492,1636083758,579,1636074180,"phú yên, tuy hòa, bánh canh hẹ, sài gòn, ẩm th...",Du khách đến Phú Yên có thể dễ dàng thưởng th...,du-lich


In [34]:
df = df.drop(columns=['Unnamed: 0', 'id', 'author', 'updatetime', 'wordcount', 'publication', 'tags'])
df.head()

,title,content,category
0,Hai thứ cần kiểm tra ngay khi nhận phòng khách...,"Nếu chăn, ga, gối và tấm trải giường nhàu nhĩ...",du-lich
1,Thảo Cầm Viên nhộn nhịp ngày đầu mở cửa trở lại,"Sáng 5/11, người dân xếp hàng ở quầy chờ mua ...",du-lich
2,Cúc Phương là vườn quốc gia hàng đầu châu Á 2021,"Để giành danh hiệu trên, Cúc Phương đã vượt q...",du-lich
3,Cuộc sống không thể thiếu chocolate ở Bỉ,"""Người Bỉ khó có thể tồn tại mà không có choc...",du-lich
4,Bánh canh hẹ Phú Yên ở Sài Gòn,Du khách đến Phú Yên có thể dễ dàng thưởng th...,du-lich


In [35]:
df = df.dropna(subset=['title', 'content'])

In [36]:
def text_normalizer(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", " ", text)
    text = re.sub("\s+", " ", text).strip()
    return text

In [37]:
def preprocess_text(text, stopwords):
    text = text_normalizer(text)
    tokens = word_tokenize(text, format='text')
    tokens = [word for word in tokens.split() if word not in stopwords]
    return ' '.join(tokens)

In [38]:
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())

df['processed_content'] = df['content'].apply(lambda x: preprocess_text(x, stopwords))

In [39]:
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [40]:
def train_topic_modeling(df, num_topics=5, model_type='lda'):
    category_models = {}
    for category in df['category'].unique():
        category_data = df[df['category'] == category]['processed_content']
        if model_type == 'lda':
            vectorizer = CountVectorizer()
            model = LatentDirichletAllocation(n_components=num_topics, random_state=0)
        elif model_type == 'nmf':
            vectorizer = TfidfVectorizer()
            model = NMF(n_components=num_topics, random_state=0)
        else:
            raise ValueError("Invalid model type. Choose either 'lda' or 'nmf'.")
        
        data_vectorized = vectorizer.fit_transform(category_data)
        model.fit(data_vectorized)
        category_models[category] = (model, vectorizer)
    return category_models

In [41]:
def train_overall_model(df, num_topics=5, model_type='lda'):
    if model_type == 'lda':
        vectorizer = CountVectorizer()
        model = LatentDirichletAllocation(n_components=num_topics, random_state=0)
    elif model_type == 'nmf':
        vectorizer = TfidfVectorizer()
        model = NMF(n_components=num_topics, random_state=0)
    else:
        raise ValueError("Invalid model type. Choose either 'lda' or 'nmf'.")
    
    data_vectorized = vectorizer.fit_transform(df['processed_content'])
    model.fit(data_vectorized)
    return model, vectorizer

In [42]:
category_models = train_topic_modeling(df, num_topics=5, model_type='lda')
overall_model, overall_vectorizer = train_overall_model(df, num_topics=5, model_type='lda')

In [43]:
import joblib

joblib.dump(category_models, 'category_models.pkl')
joblib.dump(overall_model, 'overall_model.pkl')
joblib.dump(overall_vectorizer, 'overall_vectorizer.pkl')

['overall_vectorizer.pkl']